<a href="https://colab.research.google.com/github/Approach-Analytics/Emotion-Classifier/blob/main/Train_Anger_RNN_attent_May4%2C2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Anger Training run - Last updated May 5,2023

Synchronize this notebook with the new dataset creation capacity. We also want to try to synchronize the word distribution as well... 



In [1]:
#Need pip install tensorflow with...Jupyter Notebook

!pip install nlp
#!pip install tensorflow
!pip install h5py


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 4.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd
import pickle

In [3]:
#Dec 7, 2022: Not sure what this piece of code is used for... 

import warnings
warnings.filterwarnings('ignore')

In [4]:
#Setting column options

pd.set_option('display.width', 200)
pd.set_option('display.max_columns', 100)

#Supressing the scientific notation 

pd.set_option('display.float_format', lambda x: '%.3f' % x)

#Import the Tokenizer

In [6]:
# Load the Drive helper and mount

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
#This is the Gutenberg universal tokenizer... 

path = '/content/drive/MyDrive/Sean/Emoclass/Tokenizer_Universal_125Kvoc_May5.2023.pickle'

with open(path, 'rb') as f:
    tokenizer = pickle.load(f)

#Importing inference dataset

In [ ]:
# Load inference dataset...Let's use df1 so there are no mix ups 

df1 = pd.read_csv("/content/drive/MyDrive/Sean/Twitter/COVID_223KTweets.csv")

In [ ]:
df1.columns

In [ ]:
#Create word length function... 

def count_words(text):
    return len(text.split())

In [ ]:
#Applying the function... 

df1['word_count'] = df1['tweet'].apply(count_words)

##Importing dataset and defining emotion axis 

In [ ]:
#Anger Schema - May 4,2023  
#If we call it emo_axis -> we don't have to change it across capabilities....

emo_axis = {
    "rage": ['rage', 'raging', 'raged', 'fury', 'wrath', 'wrathful', 'furious', 'enraged', 'enraging', 'incensed', 'outraged'],
    "angry": ["angry", "anger", "angered", "mad", "maddening", "maddened"],
    "frustrated": ["frustration", "frustrate", "frustrated", "frustrating", "exasperated", "exasperating", "discontented", "vexed", "vexing", "bothered", "bothersome"],
    "agitated": ["agitated", "agitating", "agitation", "aggravated", "aggravation", "aggravating", "upset", 'upsetting', 'irritate', 'irritating', 'irritable', 'irritated'],
    "annoyed": ["annoying", "annoy", "annoyed", "pestered", "pestering", "pester", "troubled", "troubling", "disturbed", "disturbing", "harassed", "harassing", "nagged", "nagging"],
    "calm": ["calm", "peaceful", "serene", "serenity", "relax", "relaxing", "relaxed"]
}


In [ ]:
#Load the dataset 

path = "/content/drive/MyDrive/Sean/Emoclass/Emotion datasets/Anger_G2000_1million_May4,2023.csv"
df=pd.read_csv(path)

In [ ]:
df.shape

In [ ]:
#Looking at the dataset 

#df.head(100)

In [ ]:
df.columns

In [ ]:
#Relabelling a column... if needed... 

df.rename(columns={"emotion2": "label"},inplace =True)

In [ ]:
#Checking to see if classes are balanced or not...

df['label'].value_counts()

#Word length distribution... 

In [ ]:
#Apply function to calculate word length

df['word_count'] = df['text'].apply(count_words)

In [ ]:
df['word_count'].describe()

#Shuffling and Truncating Distribution 

In [ ]:
# Shuffle the DataFrame
df = df.sample(frac=1, random_state=42)

# Reset the index of the shuffled DataFrame
df.reset_index(drop=True, inplace=True)


In [ ]:
# Creating a quantile list
quantile_list = [i / 100 for i in range(41)]

print(quantile_list)

# Calculate quantiles for both datasets using the quantile list
quantiles_df = pd.DataFrame({
    'Inference_dataset': df1['word_count'].quantile(quantile_list),
    'Training_dataset': df['word_count'].quantile(quantile_list)
})

# Calculate the difference between quantile values for each dataset
quantiles_df['Difference'] = quantiles_df['Training_dataset'] - quantiles_df['Inference_dataset']

# Define a function to get the quantile difference for a given word_count value
def get_quantile_diff(word_count, quantiles_df):
    for i in range(len(quantile_list) - 1):
        if word_count <= quantiles_df.loc[quantile_list[i+1], 'Training_dataset']:
            return quantiles_df.loc[quantile_list[i], 'Difference']
    return quantiles_df.loc[quantile_list[-1], 'Difference']

# Calculate 'quantile_diff' for each row in the 'df' DataFrame
df['quantile_diff'] = df['word_count'].apply(lambda x: get_quantile_diff(x, quantiles_df))

# Display the resulting dataframe
df.head()


In [ ]:
#Defining a function to remove text from both ends based on difference in values in quantiles...

def truncate_text(text, quantile_diff):
    words = text.split()
    words_to_remove = int(abs(quantile_diff))
    
    if words_to_remove >= len(words):
        return ""
    
    if quantile_diff > 0:
        # Remove words equally from the beginning and the end
        start_index = words_to_remove // 2
        end_index = -(words_to_remove - start_index)
        if end_index == 0:
            truncated_words = words[start_index:]
        else:
            truncated_words = words[start_index:end_index]
    else:
        # Add placeholder words to the beginning and the end
        placeholder = "<placeholder>"
        padding = [placeholder] * (words_to_remove // 2)
        truncated_words = padding + words + padding
    
    truncated_text = ' '.join(truncated_words)
    return truncated_text



In [ ]:
# Truncate the text in the 'text' column based on the 'quantile_diff' values
df['truncated_text'] = df.apply(lambda row: truncate_text(row['text'], row['quantile_diff']), axis=1)

# Display the resulting dataframe
df.head()

In [ ]:
#Reapply word count

df['trunc_word_counts'] = df['truncated_text'].apply(count_words)

In [ ]:
df['trunc_word_counts'].describe()

In [ ]:
import matplotlib.pyplot as plt

plt.hist(df['trunc_word_counts'], bins=200)
plt.xlabel('Word Count')
plt.ylabel('Frequency')
plt.title('Truncated Word Count Histogram')
plt.show()


# ***Reassigning Truncated Text Variable Back 

In [ ]:
df['text'] = df['truncated_text']

##Replacing the emotion words in the dataset 

In [ ]:
#Generate all the unique emotion words that then get replaced... 

a = df['emotion1'].unique().tolist()

#The list a is our list of variable responses from the dataset... 
keyword = "emotions"
words = a
for j in words: 
  df['text'] = df['text'].str.replace(j,keyword)

##Counting vocab size 

In [ ]:
from collections import Counter

# Define a function to split text into words
def split_text(text):
    return text.split()

# Split the text in the 'text' column and count the occurrences of each word
word_counter = Counter()
for text in df['text']:
    words = split_text(text)
    word_counter.update(words)

# Calculate the vocabulary size
vocab_size = len(word_counter)

print(f"The vocabulary size is: {vocab_size}")


##Splitting into train, validate and test datasets inluding shuffle dataset 

In [ ]:
#This outputs 3 different dataframes... originally 0.6 and 0.8

train, validate, test = np.split(df.sample(frac=1, random_state=42),
                       [int(.8*len(df)), int(.9*len(df))])

##Preparing Labels - Need to change emotion axis 

In [ ]:
#Converting the pandas dataframe into a list of labels... 
#We may consider puting this into a function... 

trainlabel=train['label'].tolist()
vallabel=validate['label'].tolist()
testlabel=test['label'].tolist()

In [ ]:
#Creating the classes variable... with new emotion classifier..
# loneliness_axis

classes = list(emo_axis.keys())


In [ ]:
# Map each class to a unique integer
classes_to_index = dict((c, i) for i, c in enumerate(classes))

# Map each integer back to its corresponding class
index_to_classes = dict((v, k) for k, v in classes_to_index.items())

In [ ]:
#Creating a lambda function...called "names_to_ids"

names_to_ids = lambda labels: np.array([classes_to_index.get(x) for x in labels])

In [ ]:
#Applying the names_to_ids functions to the labels

train_labels = names_to_ids(trainlabel)
val_labels = names_to_ids(vallabel)
test_labels = names_to_ids(testlabel)

#Testing out the labels...
print(train_labels[10])

In [ ]:
#What is train_labels?

train_labels?

In [ ]:
import numpy as np

# Use numpy's unique() and return_counts=True parameters to get value counts
values, counts = np.unique(train_labels, return_counts=True)

# Create a dictionary with the values and their respective counts
value_counts = dict(zip(values, counts))

# Print the dictionary
print(value_counts)


##Input Training Text and Tokenizing Tweets

In [ ]:
#Converting the different pandas dataframes into a list of text fields...
#Choice of variables: truncated_text, text_minus1, 'filtered_text',''filtered_text_minus1'

traintext=train['text'].tolist()
valtext=validate['text'].tolist()
testtext=test['text'].tolist()

##Trying "universal" tokenizer - trained on Gutenberg

In [ ]:
#Importing the tokenizer...
#Input into the tokenizer is a list

#from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
#Input into the tokenizer is a list

#tokenizer = Tokenizer(num_words=131000, oov_token='<UNK>')

#I think that this is the missing piece...I'm not really sure what it does...  

#tokenizer.fit_on_texts(traintext)
#tokenizer.fit_on_texts(valtext)

#Testing the tokenization... 

#print(tokenizer.texts_to_sequences([tweets[10]]))

#Tokenizing and padding 

##Pay attention to where tokenizer coming from...

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
#Creating a function that tokenizes and pads the sequences...
#Max word length is 65...

def get_sequences(tokenizer, texts):
    sequences = tokenizer.texts_to_sequences(texts)
    padded_sequences = pad_sequences(sequences, truncating='post', maxlen=65, padding='post')
    return padded_sequences

In [ ]:
#Applying the function to tokenize and pad... to all test,validate and test
#Syntax: val_sequences = get_sequences(tokenizer, val_tweets)

padded_train_sequences = get_sequences(tokenizer, traintext)
val_sequence = get_sequences(tokenizer,valtext)
test_sequence = get_sequences(tokenizer,testtext)


#Adding attention layer and specifying the model 

In [ ]:
from tensorflow.keras.layers import Layer

class Attention(Layer):
    def __init__(self, **kwargs):
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1), initializer="normal")
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1), initializer="zeros")
        super(Attention, self).build(input_shape)

    def call(self, x):
        e = tf.keras.backend.tanh(tf.keras.backend.dot(x, self.W) + self.b)
        a = tf.keras.backend.softmax(e, axis=1)
        output = x * a
        return tf.keras.backend.sum(output, axis=1)


In [ ]:
#Specifying the model - words: 76690

model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(76990, 200, input_length=65),
    tf.keras.layers.Conv1D(200, 5, activation='relu'),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(200, return_sequences=True)),
    Attention(),
    tf.keras.layers.Dense(6, activation='softmax')
])

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()


##Running the model


In [ ]:
#Most up to date architecture: April 15, 2023 -> Fa

#Having the callbacks option on means that the model stops once you are like over-generalizing... 
#Let's leave the callback option on for now...


h = model.fit(
    padded_train_sequences, train_labels,
    validation_data=(val_sequence, val_labels),
    epochs=5,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=2)
   ]
)

#Evaluating The Model 

In [ ]:
def show_history(h):
    epochs_trained = len(h.history['loss'])
    plt.figure(figsize=(16, 6))

    plt.subplot(1, 2, 1)
    plt.plot(range(0, epochs_trained), h.history.get('accuracy'), label='Training')
    plt.plot(range(0, epochs_trained), h.history.get('val_accuracy'), label='Validation')
    plt.ylim([0., 1.])
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(range(0, epochs_trained), h.history.get('loss'), label='Training')
    plt.plot(range(0, epochs_trained), h.history.get('val_loss'), label='Validation')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

show_history(h)

In [ ]:
#Running the model on the test sequence and test labels... 

eval = model.evaluate(test_sequence, test_labels)

In [ ]:
#preds = model.predict_classes(test_sequence)
preds=model.predict(test_sequence) 
classes_x=np.argmax(preds,axis=1)
preds.shape, test_labels.shape

##Error Analysis 

In [ ]:
#Dumb luck metric 

counts = df['label'].value_counts()
dumb_luck = max(counts) / sum(counts)

dumb_luck


In [ ]:
#Creating an inference dataframe 

inf_df=pd.DataFrame({
    'data':testtext,
    "labels_predicted": classes_x                    
})
inf_df["labels_predicted_marked"]=inf_df['labels_predicted'].apply(lambda x: index_to_classes[x])
inf_df["actual_labels"]=testlabel

#Creating the labels index datastructure...

inf_df["actual_label_index"]=inf_df['actual_labels'].apply(lambda x: classes_to_index[x])



In [ ]:
#Checking the inf_df shape...

inf_df.shape

In [ ]:
#Making the correct prediction 

inf_df['correct_pred']=0 # first assigning all to 0.
inf_df.loc[(inf_df['labels_predicted']==inf_df['actual_label_index']),'correct_pred']=1 # labelling 1 if the prediction is right.

In [ ]:
# magnitutde of error
inf_df['error_magnitude']=abs(inf_df['labels_predicted']-inf_df['actual_label_index'])

In [ ]:
# Count the frequency of each class in the error_magnitude column
counts = inf_df['error_magnitude'].value_counts()

# Create a histogram with one bar for each class
plt.bar(counts.index, counts.values)

# Set the title and axis labels
plt.suptitle('Histogram of Error Magnitude in Fear Classification')
plt.xlabel('Magnitude')
plt.ylabel('Frequency')

##Saving the model, tokenizer and evaluation datasets 

In [ ]:
from keras.models import load_model

# Make sure you provide the full file path, including the .h5 extension

model.save('/content/drive/MyDrive/Sean/Emoclass/Model_Anger_1mil_dist_May5,2023.h5')


In [ ]:
#Exporting the dataset 

#path = "/content/drive/MyDrive/Sean/Emoclass/Train_Fear_Error_Analysis_151K_March8,2023.csv"
inf_df.to_csv(path)
